In [20]:
### 19th March,2024
#### 20th March,2024

In [21]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [22]:

import pandas as pd 
import pybedtools

path1=('S010.xlsx')
colnames1=('chr','x1','x2')
df1=pd.read_excel('S010.xlsx',names=colnames1)

path2=('HPVint.xlsx')
colnames2=('sample ID', 'virus type', 'virus start','virus stop', 'Chr.','chr. start', 'chr. stop', 'strand1', 'strand2', '#supporting pairs', '#split read support', 'representative break', 'nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)')
ori_df2=pd.read_excel('HPVint.xlsx',names=colnames2)
ori_df2

# First, let's filter df2 to include only those rows where 'representative break' equals 'Y'
filtered_df = ori_df2[ori_df2['representative break'] == 'Y']
filtered_df




# Assuming filtered_df is the DataFrame after applying your conditions related to 'representative break'
df2 = filtered_df[['Chr.', 'chr. start', 'chr. stop', 'sample ID']]
df2 = df2.reset_index(drop=True)
df2

Chr.  chr. start  chr. stop sample ID
0    chr17     1104459    1104460   GS18001
1     chr2     9961582    9961583   GS18001
2    chr21    11122389   11122390   GS18001
3     chr5    29687633   29687634   GS18001
4     chr5    91748270   91748271   GS18001
..     ...         ...        ...       ...
313   chr9     5437959    5437960   GS21051
314   chr1   226954998  226954999   GS21052
315  chr10    18916383   18916384   GS21052
316   chr3     2140200    2140201   GS21052
317   chr8   129783851  129783871   GS21052

[318 rows x 4 columns]

In [23]:
import pybedtools
from pybedtools import BedTool

# Harmonize chromosome naming in df1 to add 'chr' prefix if missing
df1['chr'] = df1['chr'].apply(lambda x: 'chr' + str(x) if not str(x).startswith('chr') else str(x))

# Do the same for df2
df2['Chr.'] = df2['Chr.'].apply(lambda x: 'chr' + str(x) if not str(x).startswith('chr') else str(x))

bed1 = BedTool.from_dataframe(df1[['chr', 'x1', 'x2']])
bed2 = BedTool.from_dataframe(df2[['Chr.', 'chr. start', 'chr. stop']])



#  Sort the BedTool objects 
sorted_bed1 = bed1.sort()
sorted_bed2 = bed2.sort()

overlap = sorted_bed1.intersect(sorted_bed2, wa=True, wb=True)

# To save the overlap results to a file
overlap.saveas('TAD_HPV_integration_overlap_S010.bed')
# convert to DataFrame
overlap_df = overlap.to_dataframe(names=['TAD_chr', 'TAD_start', 'TAD_end', 
                                         'HPV_chr', 'HPV_start', 'HPV_end'])

print(overlap_df)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
0     chr1    9330000    9540000    chr1    9435159    9435160
1     chr1    9640000   10490000    chr1   10179462   10179463
2     chr1   15850000   16710000    chr1   16245129   16245130
3     chr1   15850000   16710000    chr1   16461227   16461228
4     chr1   47900000   48180000    chr1   47900293   47900294
..     ...        ...        ...     ...        ...        ...
68    chr9  101940000  103080000    chr9  102720378  102720379
69    chr9  108590000  109940000    chr9  109336186  109336187
70    chr9  110220000  111340000    chr9  110354690  110354691
71    chr9  131890000  132370000    chr9  131937709  131937710
72    chrX    1730000    2550000    chrX    2211164    2211165

[73 rows x 6 columns]


In [24]:
### Identifying integrations in multiple TADs and merging those TADs in a single TAD

grouped = overlap_df.groupby(['HPV_chr', 'HPV_start', 'HPV_end'])

#### Aggregate to find the minimum and maximum TAD coordinates for each HPV integration


# Aggregate to find the new TAD boundaries
new_tad_boundaries = grouped.agg(TAD_chr=('TAD_chr', 'first'), 
                                 new_TAD_start=('TAD_start', 'min'), 
                                 new_TAD_end=('TAD_end', 'max')).reset_index()

# This renaming aligns with your intended updates without requiring a suffix
new_tad_boundaries.rename(columns={'new_TAD_start': 'TAD_start_new', 'new_TAD_end': 'TAD_end_new'}, inplace=True)
merged_info = pd.merge(overlap_df, new_tad_boundaries, 
                       on=['HPV_chr', 'HPV_start', 'HPV_end'], 
                       how='left', 
                       suffixes=('', '_new'))
# Directly update 'TAD_start' and 'TAD_end' in 'overlap_df' with the new values
merged_info['TAD_start'] = merged_info['TAD_start_new']
merged_info['TAD_end'] = merged_info['TAD_end_new']

# Drop the temporary new columns used for the update
merged_info.drop(['TAD_start_new', 'TAD_end_new'], axis=1, inplace=True)
merged_info

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end TAD_chr_new
0     chr1    9330000    9540000    chr1    9435159    9435160        chr1
1     chr1    9640000   10490000    chr1   10179462   10179463        chr1
2     chr1   15850000   16710000    chr1   16245129   16245130        chr1
3     chr1   15850000   16710000    chr1   16461227   16461228        chr1
4     chr1   47900000   48180000    chr1   47900293   47900294        chr1
..     ...        ...        ...     ...        ...        ...         ...
68    chr9  101940000  103080000    chr9  102720378  102720379        chr9
69    chr9  108590000  109940000    chr9  109336186  109336187        chr9
70    chr9  110220000  111340000    chr9  110354690  110354691        chr9
71    chr9  131890000  132370000    chr9  131937709  131937710        chr9
72    chrX    1730000    2550000    chrX    2211164    2211165        chrX

[73 rows x 7 columns]

In [25]:
####Check if 'TAD_chr_new' is identical to 'TAD_chr' in all rows
columns_are_identical = (merged_info['TAD_chr'] == merged_info['TAD_chr_new']).all()

print(f"Is 'TAD_chr_new' identical to 'TAD_chr' in all rows? {columns_are_identical}")

Is 'TAD_chr_new' identical to 'TAD_chr' in all rows? True


In [26]:
merged_info.drop('TAD_chr_new', axis=1, inplace=True)
merged_info

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
0     chr1    9330000    9540000    chr1    9435159    9435160
1     chr1    9640000   10490000    chr1   10179462   10179463
2     chr1   15850000   16710000    chr1   16245129   16245130
3     chr1   15850000   16710000    chr1   16461227   16461228
4     chr1   47900000   48180000    chr1   47900293   47900294
..     ...        ...        ...     ...        ...        ...
68    chr9  101940000  103080000    chr9  102720378  102720379
69    chr9  108590000  109940000    chr9  109336186  109336187
70    chr9  110220000  111340000    chr9  110354690  110354691
71    chr9  131890000  132370000    chr9  131937709  131937710
72    chrX    1730000    2550000    chrX    2211164    2211165

[73 rows x 6 columns]

In [27]:
# Ensure both DataFrames are aligned (same index and sorted in the same order if necessary)
# For this comparison, the indices of the rows being compared should correspond to the same data points

# Create a boolean mask for rows where 'TAD_start' or 'TAD_end' are different between the two DataFrames
diff_mask = (merged_info['TAD_start'] != overlap_df['TAD_start']) | (merged_info['TAD_end'] != overlap_df['TAD_end'])
 
# Apply the mask to get the rows from merged_info where 'TAD_start' or 'TAD_end' are different
rows_with_diff = merged_info[diff_mask]

print(rows_with_diff)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
30   chr14  100850000  102130000   chr14  101355673  101355674


In [28]:
# Using the previously created diff_mask to select rows from overlap_df
original_rows_with_diff = overlap_df[diff_mask]

print(original_rows_with_diff)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
30   chr14  101270000  102100000   chr14  101355673  101355674


In [29]:
merged_info=merged_info.drop_duplicates()
merged_info

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
0     chr1    9330000    9540000    chr1    9435159    9435160
1     chr1    9640000   10490000    chr1   10179462   10179463
2     chr1   15850000   16710000    chr1   16245129   16245130
3     chr1   15850000   16710000    chr1   16461227   16461228
4     chr1   47900000   48180000    chr1   47900293   47900294
..     ...        ...        ...     ...        ...        ...
68    chr9  101940000  103080000    chr9  102720378  102720379
69    chr9  108590000  109940000    chr9  109336186  109336187
70    chr9  110220000  111340000    chr9  110354690  110354691
71    chr9  131890000  132370000    chr9  131937709  131937710
72    chrX    1730000    2550000    chrX    2211164    2211165

[72 rows x 6 columns]

In [30]:
# Filter overlap_df for rows matching the specific HPV integration criteria chr14  101270000  102100000   chr14  101355673  101355674
specific_integration = overlap_df[
    (overlap_df['HPV_chr'] == 'chr14') & 
    (overlap_df['HPV_start'] == 101355673) & 
    (overlap_df['HPV_end'] == 101355674)
]

print(specific_integration)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
29   chr14  100850000  102130000   chr14  101355673  101355674
30   chr14  101270000  102100000   chr14  101355673  101355674


In [31]:
import pandas as pd

# Assuming your DataFrame is named df

# Identify unique TAD regions and assign a unique ID to each
unique_tads = merged_info[['TAD_chr', 'TAD_start', 'TAD_end']].drop_duplicates().reset_index(drop=True)
unique_tads['TAD_id'] = ['TAD' + str(i).zfill(4) for i in range(221, 221 + len(unique_tads))]

# Create a mapping from each unique TAD region to its TAD ID
tad_id_mapping = unique_tads.set_index(['TAD_chr', 'TAD_start', 'TAD_end'])['TAD_id'].to_dict()

# Map the TAD IDs back to the original DataFrame based on the TAD region
merged_info['TAD_id'] = merged_info.apply(lambda row: tad_id_mapping[(row['TAD_chr'], row['TAD_start'], row['TAD_end'])], axis=1)

# Display the DataFrame to verify the TAD_id assignment
print(merged_info)


   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end   TAD_id
0     chr1    9330000    9540000    chr1    9435159    9435160  TAD0221
1     chr1    9640000   10490000    chr1   10179462   10179463  TAD0222
2     chr1   15850000   16710000    chr1   16245129   16245130  TAD0223
3     chr1   15850000   16710000    chr1   16461227   16461228  TAD0223
4     chr1   47900000   48180000    chr1   47900293   47900294  TAD0224
..     ...        ...        ...     ...        ...        ...      ...
68    chr9  101940000  103080000    chr9  102720378  102720379  TAD0279
69    chr9  108590000  109940000    chr9  109336186  109336187  TAD0280
70    chr9  110220000  111340000    chr9  110354690  110354691  TAD0281
71    chr9  131890000  132370000    chr9  131937709  131937710  TAD0282
72    chrX    1730000    2550000    chrX    2211164    2211165  TAD0283

[72 rows x 7 columns]


/tmp/ipykernel_13198/1951769995.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_info['TAD_id'] = merged_info.apply(lambda row: tad_id_mapping[(row['TAD_chr'], row['TAD_start'], row['TAD_end'])], axis=1)


In [36]:
# Merging filtered_df with merged_info_df_S009 to associate sample ID and nearby genes with TADs
merged_info_with_samples = pd.merge(merged_info, 
                                    filtered_df[['Chr.', 'chr. start', 'chr. stop', 'sample ID', 'nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)']],
                                    left_on=['HPV_chr', 'HPV_start', 'HPV_end'], 
                                    right_on=['Chr.', 'chr. start', 'chr. stop'],
                                    how='left')

merged_info_with_samples
merged_info_with_samples=merged_info_with_samples.drop_duplicates()
merged_info_with_samples

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end   TAD_id  Chr.  \
0     chr1    9330000    9540000    chr1    9435159    9435160  TAD0221  chr1   
1     chr1    9640000   10490000    chr1   10179462   10179463  TAD0222  chr1   
2     chr1   15850000   16710000    chr1   16245129   16245130  TAD0223  chr1   
3     chr1   15850000   16710000    chr1   16461227   16461228  TAD0223  chr1   
4     chr1   47900000   48180000    chr1   47900293   47900294  TAD0224  chr1   
..     ...        ...        ...     ...        ...        ...      ...   ...   
67    chr9  101940000  103080000    chr9  102720378  102720379  TAD0279  chr9   
68    chr9  108590000  109940000    chr9  109336186  109336187  TAD0280  chr9   
69    chr9  110220000  111340000    chr9  110354690  110354691  TAD0281  chr9   
70    chr9  131890000  132370000    chr9  131937709  131937710  TAD0282  chr9   
71    chrX    1730000    2550000    chrX    2211164    2211165  TAD0283  chrX   

    chr. start  chr. stop     sample ID  \
0      9435159    9435160       GS18051   
1     10179462   10179463       GS18010   
2     16245129   16245130       GS21009   
3     16461227   16461228       GS18005   
4     47900293   47900294       GS18092   
..         ...        ...           ...   
67   102720378  102720379  TCGA-BA-A4IH   
68   109336186  109336187       GS18026   
69   110354690  110354691       GS18010   
70   131937709  131937710       GS18062   
71     2211164    2211165       GS18001   

   nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)  
0                    SPSB1,LOC100506022,H6PD,SLC25A33                                 
1                             UBE4B,KIF1B,RBP7,NMNAT1                                 
2                       SPEN, ZBTB17, FLJ37453, SRARP                                 
3                       EPHA2,FAM131C,ARHGEF19,CLCNKB                                 
4                     LINC01389,FOXD2-AS1,FOXD2,FOXE3                                 
..                                                ...                                 
67                        STX17,ERP44,STX17-AS1,NR4A3                                 
68                 MIR8081,LINC01505,ZNF462,LOC340512                                 
69                    KLF4,LINC01509,RAD23B,LOC340512                                 
70                             IER5L,PTPA,CRAT,DOLPP1                                 
71                         DHRSX,ZBED1,MIR6089,CD99P1                                 

[72 rows x 12 columns]

In [37]:
# Count unique samples per TAD
import pandas as pd
sample_counts_per_TAD = merged_info_with_samples.groupby('TAD_id')['sample ID'].nunique()

# Identify TAD IDs with at least 3 samples
tads_of_interest_ids = sample_counts_per_TAD[sample_counts_per_TAD >= 3].index

# Filter for TADs of interest based on the identified TAD IDs
tads_of_interest = merged_info_with_samples[merged_info_with_samples['TAD_id'].isin(tads_of_interest_ids)]
if not tads_of_interest_ids.empty:
    print("TADs of interest based on having at least 3 samples:")
    print(tads_of_interest_ids)
else:
    print("No TAD of interest is found")

TADs of interest based on having at least 3 samples:
Index(['TAD0242', 'TAD0269', 'TAD0278'], dtype='object', name='TAD_id')


In [38]:
merged_info_with_samples.to_excel('merged_info_with_samples_S010.xlsx')